# Working with uDALES Field Data

This tutorial describes how to read and process field data output of the LES code uDALES using Python. It covers important concepts such as:
- Grid layout and variable locations
- Averaging procedures used in uDALES output
- Loading different types of field data

The **UDBase** post-processing class contains methods to load field data:
- **load_stat_xyt**: Time- and slab-averaged statistics (xytdump.expnr.nc)
- **load_stat_t**: Time-averaged statistics (tdump.expnr.nc)
- **load_field**: Instantaneous 3D data (fielddump.expnr.nc)
- **load_slice**: Instantaneous 2D slices (Xslicedump.expnr.nc)

## 1. Import Libraries and Setup

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Add uDALES python tools to path
udales_root = Path.cwd().parent.parent if 'docs' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(udales_root / 'tools' / 'python'))

from udbase import UDBase

# Initialize UDBase with experiment directory
expnr = '110'
expdir = Path('../experiments/110')

# Check if directory exists
if expdir.exists():
    sim = UDBase(case_dir=str(expdir))
    print(f"✓ Initialized UDBase for experiment {expnr}")
else:
    print(f"⚠ Directory {expdir} not found - adjust path as needed")
    sim = None

## 2. uDALES Grid Layout

uDALES uses a **staggered grid** - not all variables are defined at the same location. This is computationally advantageous but requires care with plotting.

### Grid Staggering (x-z view)

```
                      w(i,j,k+1)
                          ^
zm(k+1) --        --------|--------
                  |               |
                  |   c(i,j,k)    |
  zt(k) --  u(i,j,k) --> o      ---> u(i+1,j,k)
                  |               |
                  |       ^       |
  zm(k) --        --------|--------
                      w(i,j,k)
                  |       |       |
                xm(i)   xt(i)   xm(i+1)
```

### Coordinate Definitions

- **xm(i)** = (i-1) × dx;  **xt(i)** = (i-1/2) × dx
- **ym(j)** = (j-1) × dy;  **yt(j)** = (j-1/2) × dy
- **zm(k)** = (k-1) × dz;  **zt(k)** = (k-1/2) × dz

### Grid Increments

- **dzt(k)** = zm(k+1) - zm(k) : cell height
- **dzm(k)** = zt(k) - zt(k-1) : distance between cell centers

In [ ]:
if sim is not None:
    print(f"Grid dimensions: {sim.itot} × {sim.jtot} × {sim.ktot}")
    print(f"Domain size: {sim.xlen if hasattr(sim, 'xlen') else 'N/A'} × "
          f"{sim.ylen if hasattr(sim, 'ylen') else 'N/A'} × "
          f"{sim.zsize if hasattr(sim, 'zsize') else 'N/A'} m")
    print(f"Grid spacing: dx={sim.dx}, dy={sim.dy}")
    print()
    print("Available coordinate arrays:")
    print(f"  xm, xt: x-coordinates (edges/centers)")
    print(f"  ym, yt: y-coordinates (edges/centers)")
    print(f"  zm, zt: z-coordinates (edges/centers)")
else:
    print("Example: Grid dimensions: 64 × 64 × 64")
    print("Domain size: 64 × 64 × 64 m")

## 3. load_stat_xyt: Time- and Slab-Averaged Data

This method loads 1D vertical profiles that are averaged in x, y, and time from `xytdump.expnr.nc`.

In [ ]:
# Load and plot mean velocity profile
if sim is not None:
    try:
        # Load time- and slab-averaged streamwise velocity
        uxyt = sim.load_stat_xyt('uxyt', time=None)
        print(f"✓ Loaded uxyt with shape: {uxyt.shape}")
        
        # Create plot
        plt.figure(figsize=(8, 6))
        
        # Plot profile(s)
        if len(uxyt.shape) > 1 and uxyt.shape[-1] > 1:
            for n in range(min(uxyt.shape[-1], 5)):  # Plot up to 5 times
                plt.plot(uxyt[:, n], sim.zm, linewidth=2, label=f'Interval {n+1}')
        else:
            plt.plot(uxyt, sim.zm, linewidth=2)
        
        plt.ylabel('z [m]')
        plt.xlabel('⟨ū⟩ [m/s]')
        plt.title('Mean Streamwise Velocity Profile')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        # plt.show()  # Uncomment to display
        print("✓ Plot created")
        
    except Exception as e:
        print(f"Note: {e}")
        print("Example output shown when data is available")
else:
    print("Example usage:")
    print("  uxyt = sim.load_stat_xyt('uxyt', time=None)")
    print("  plt.plot(uxyt, sim.zm)")

## 4. Momentum Flux Analysis

uDALES separates momentum fluxes into turbulent and dispersive components:
- **Turbulent flux**: ⟨u'w'⟩ (from upwpxyt)
- **Dispersive flux**: ⟨u"w"⟩ (from uwxyt)
- **Total flux**: ⟨u'w'⟩ + ⟨u"w"⟩

In [ ]:
# Example momentum flux analysis
print("Example workflow for momentum flux analysis:")
print()
print("# Load flux components")
print("upwpxyt = sim.load_stat_xyt('upwpxyt')  # Turbulent flux")
print("uwxyt = sim.load_stat_xyt('uwxyt')      # Dispersive flux")
print()
print("# Calculate total flux")
print("total_flux = upwpxyt + uwxyt")
print()
print("# Plot comparison")
print("plt.plot(upwpxyt, sim.zt, label='Turbulent')")
print("plt.plot(uwxyt, sim.zt, label='Dispersive')")
print("plt.plot(total_flux, sim.zt, label='Total', linewidth=2)")
print("plt.ylabel('z [m]')")
print("plt.xlabel('⟨uw⟩ [m²/s²]')")
print("plt.legend()")
print()
print("Note: Flux analysis requires specific output configuration.")

## 5. load_stat_t: Time-Averaged 3D Data

Loads 3D time-averaged statistics from `tdump.expnr.nc`. Returns xarray DataArray with dimensions (z, y, x).

In [ ]:
# Example: Load time-averaged 3D data
print("Usage:")
print("  T_avg = sim.load_stat_t('T')   # Temperature")
print("  u_avg = sim.load_stat_t('u')   # Velocity")
print()
print("Returns: xarray DataArray (z, y, x)")
print()
print("Example analysis:")
print("""
# Load data
T_avg = sim.load_stat_t('T')

# Extract horizontal slice at height z=20m
z_idx = np.argmin(np.abs(sim.zt - 20))
T_slice = T_avg[z_idx, :, :]

# Plot
plt.contourf(sim.xt, sim.yt, T_slice, levels=20)
plt.colorbar(label='Temperature [K]')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.title('Time-averaged Temperature at z=20m')
""")

## 6. load_field: Instantaneous 3D Data

Loads instantaneous 3D fields from `fielddump.expnr.nc`. Can load specific time or all times.

In [ ]:
# Example: Load instantaneous field data
print("Usage:")
print("  # Load at specific time")
print("  u = sim.load_field('u', time=3600)")
print()
print("  # Load all times")
print("  u_all = sim.load_field('u')")
print()
print("Returns:")
print("  Single time: DataArray (z, y, x)")
print("  All times: DataArray (time, z, y, x)")
print()
print("Example analysis:")
print("""
# Load velocity at t=3600s
u = sim.load_field('u', time=3600)

# Horizontal slice at z=10m
z_idx = np.argmin(np.abs(sim.zt - 10))
u_slice = u[z_idx, :, :]

# Vertical profile (domain average)
u_profile = u.mean(dim=['y', 'x'])

# Plot horizontal slice
plt.figure(figsize=(10, 4))
plt.subplot(121)
plt.contourf(sim.xt, sim.yt, u_slice, levels=20)
plt.colorbar(label='u [m/s]')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.title(f'u at z={sim.zt[z_idx]:.1f}m')

# Plot vertical profile
plt.subplot(122)
plt.plot(u_profile, sim.zt)
plt.xlabel('⟨u⟩ [m/s]')
plt.ylabel('z [m]')
plt.title('Domain-averaged u profile')
""")

## 7. load_slice: 2D Slice Data

Loads instantaneous 2D slices from slice output files. Available slice types: 'xy', 'xz', 'yz'.

In [ ]:
# Example: Load 2D slice data
print("Usage:")
print("  # XY slice at constant z")
print("  u_xy = sim.load_slice('xy', 'u', time=3600)")
print()
print("  # XZ slice at constant y")
print("  u_xz = sim.load_slice('xz', 'u', time=3600)")
print()
print("  # YZ slice at constant x")
print("  u_yz = sim.load_slice('yz', 'u', time=3600)")
print()
print("Available slice types: 'xy', 'xz', 'yz'")
print()
print("Example visualization:")
print("""
# Load XZ slice
u_xz = sim.load_slice('xz', 'u', time=3600)

# Plot
plt.contourf(sim.xt, sim.zt, u_xz.T, levels=20)
plt.colorbar(label='u [m/s]')
plt.xlabel('x [m]')
plt.ylabel('z [m]')
plt.title('Streamwise velocity (XZ slice)')
""")

## 8. Averaging Procedures in uDALES

uDALES uses several types of averaging to analyze turbulent urban flows:

### 1. Slab Averaging (⟨·⟩)
Average in x and y directions, produces vertical profiles:
- Used in `load_stat_xyt`
- Example: ⟨u⟩(z, t)

### 2. Time Averaging (‾)
Average over time interval, reduces statistical noise:
- Used in all stat outputs
- Example: ū(x, y, z)

### 3. Reynolds Decomposition
Separates instantaneous field into mean and fluctuating components:
- φ = φ̄ + φ'
- φ̄: mean component
- φ': fluctuating component

### 4. Dispersive Decomposition
Separates slab-averaged mean into uniform and spatially-varying parts:
- φ̄ = ⟨φ̄⟩ + φ̄"
- ⟨φ̄⟩: slab-averaged mean
- φ̄": spatial deviation from slab average

### Key Relationships
- **Turbulent flux**: ⟨u'w'⟩ (from upwpxyt)
- **Dispersive flux**: ⟨ū"w̄"⟩ (from uwxyt)
- **Total flux**: ⟨u'w'⟩ + ⟨ū"w̄"⟩

## 9. Summary

### Key Takeaways
- ✅ UDBase provides unified interface for all field data types
- ✅ Grid is staggered - pay attention to variable locations (u, v, w at edges; scalars at centers)
- ✅ xarray DataArrays provide labeled dimensions for easy manipulation
- ✅ Methods handle file I/O and coordinate management automatically

### Data Loading Methods
| Method | File | Description | Dimensions |
|--------|------|-------------|------------|
| `load_stat_xyt` | xytdump | Time & slab averaged | (z, time) |
| `load_stat_t` | tdump | Time averaged 3D | (z, y, x) |
| `load_field` | fielddump | Instantaneous 3D | (z, y, x) or (time, z, y, x) |
| `load_slice` | *slicedump | Instantaneous 2D | (dim1, dim2) |

### Next Steps
- **facets_tutorial.ipynb** - Surface data analysis and energy balance
- **geometry_tutorial.ipynb** - Creating and manipulating urban geometries
- `tools/python/fields_example.py` - Complete working examples
- `tools/python/QUICK_REFERENCE.py` - API reference guide